# グラスウール 16K
# （glass wool 16K）
参考：熱湿気計算と物性値WG用  
[1]  齋藤宏昭、土屋喬雄：冷房時における夏型壁体内結露に関する研究、その１　人工気候室における非定常結露実験、日本建築学会計画系論文集　第519号、pp39-45、1999年5月  
[2] 土屋喬雄、黒木勝一、齋藤宏昭：冷房時の夏型壁体内結露に関する研究 （その 3） 1次元モデルの検証と壁体内結露の長期予測、日本建築学会大会学術講演梗概集 pp.269-270、1996年9月  
[3]　独立行政法人　建築研究所、カナダ天然資源省　編：第3回　日加住宅R&D専門家作業会合　「断熱外皮の設計」講演予稿集


In [1]:
module glass_wool_16K

# 空隙率
const psi = 0.99 # おおよその値

# 材料密度
const row = 16.0 #kg/m3

# 比熱
const C = 698.0 #J/kg

# 水の密度
const roww = 1000.0 #kg/m3
#理想気体定数
const R = 8.314 # J/(mol K)
# 水のモル質量
const Mw = 0.018 # kg/mol
# 水蒸気のガス定数
Rv = R / Mw # J/(kg K)
# 水の熱容量
const croww = 1000.0 * 4.18605e+3

# miu ⇒　rh の変換係数
function convertMiu2RH( ;temp::Float64, miu::Float64 );
    return exp( miu / Rv / temp )
end

# 熱容量
get_crow( ;phi::Float64 ) = C * row + croww * phi
get_crow( cell ) = get_crow( phi = get_phi( cell ) )

### 水分特性 ###
# 含水率
# 関数系　phi = a * ( b / ( rh + c ) + d ) [kg/kg]
# 相対湿度の閾値 rh_TH
a = [ 1.0, 1.0 ]
b = [ -0.00451762, -0.00000906546 ]
c = [ 2.86089, -1.00131 ]
d = [ 0.00157909, 0.000302253 ]
rh_TH = [ 0.8357 ]


function get_phi( ;rh::Float64 )
    if rh < rh_TH[1]
        phi = ( a[1] * ( b[1] / ( rh + c[1] ) + d[1] ) )
    elseif rh >= rh_TH[1] && rh <= 0.99 #0.999
        phi = ( a[2] * ( b[2] / ( rh + c[2] ) + d[2] ) )
    # 髙取追記：2024/5/09
    # 結露判定用
    elseif rh > 0.99
        phi = -1.9777924016366822e6 * rh^3 + 5.903710318859591e6 * rh^2 -5.874043432809136e6 * rh + 1.9481265055862272e6
    end
    return phi
end

get_phi( cell ) = get_phi( rh = convertMiu2RH( temp = cell.temp, miu = cell.miu ) )

# 含水率の水分化学ポテンシャル微分
drh_dmiu( temp::Float64, miu::Float64 ) = ( 1.0 / Rv / temp ) * exp( miu / Rv / temp )
# dphi/dmiu = dphi/drh * drh/dmiu
function get_dphi( ;temp::Float64, miu::Float64 )
    rh = convertMiu2RH( temp = temp, miu = miu )
    if rh < rh_TH[1]
        dphi_drh = - ( a[1] * ( b[1] / (( rh + c[1] )^2) ) )
    elseif rh >= rh_TH[1] && rh <= 0.99 #0.999
        dphi_drh = - ( a[2] * ( b[2] / (( rh + c[2] )^2) ) )
    # 髙取追記：2024/3/18
    #elseif rh >= 0.999
    elseif rh > 0.99
        dphi_drh = - 3.0 * 1.9777924016366822e6 * rh^2 + 2.0 * 5.903710318859591e6 * rh - 5.874043432809136e6
    end
    return dphi_drh * drh_dmiu( temp, miu )
end
get_dphi( cell ) = get_dphi( temp = cell.temp, miu = cell.miu )

# 含水率から水分化学ポテンシャルの算出
function get_miu_by_phi( ;temp::Float64, phi::Float64 )
    if phi < get_phi( rh = rh_TH[1])
        rh = b[1] / (phi / a[1] - d[1]) - c[1]
    elseif phi >= get_phi( rh = rh_TH[1]) && phi <= 0.0011 #0.0042267
        rh = b[2] / (phi / a[2] - d[2]) - c[2]
    # 髙取追記：2024/3/18
    elseif rh >= 0.0011 #0.0042267
        #rh = ( phi - 0.0042267 ) / 980.0 + 0.999
        rh = ( phi - 0.0011 ) / 98.0 + 0.99
    end
    return Rv * temp * log( rh )
end
get_miu_by_phi( cell ) = get_miu_by_phi( temp = cell.temp, phi = get_phi( cell ) )

### 移動特性 ###
# 熱伝導率
const lam = 0.045

# 湿気依存
get_lam( ;phi::Float64 ) = lam
get_lam( cell ) = get_lam( phi = get_phi( cell ) )

# 透水係数
get_dw( ;phi::Float64 ) = 0.0
get_dw( cell ) = 0.0

# 透湿率（湿気伝導率）
# kg / kg DA
dah_dpv(pv::Float64) = 0.622 / ( 101325.0 - pv ) + 0.622 * pv / ( ( 101325.0 - pv )^2 )
dah_dpv() = dah_dpv(1700.0)
get_dp() = 2.52e-5 * dah_dpv() #絶乾時の透湿率[kg/m Pa s]
get_dp( cell ) = get_dp()

end

Main.glass_wool_16K

In [2]:
( glass_wool_16K.get_phi(rh = 0.999) - glass_wool_16K.get_phi(rh = 0.6) ) /0.1

8.828262447549337

In [15]:
glass_wool_16K.get_dphi(temp = 293.15, miu = -13500.0) / glass_wool_16K.drh_dmiu( 293.15, -13500.0 )

0.0009795099562955457

In [12]:
glass_wool_16K.get_dphi(temp = 293.15, miu = -135.0) / glass_wool_16K.drh_dmiu( 293.15, -135.0 )a


98.0

In [11]:
glass_wool_16K.get_phi(rh = 0.99)

0.0011037967665782542

In [5]:
glass_wool_16K.get_phi(rh = 1.0)

0.9811000000000009

In [6]:
glass_wool_16K.get_dp()

1.600187483489686e-10